In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
test=pd.read_csv(r"/kaggle/input/tabular-playground-series-feb-2021/test.csv")
train=pd.read_csv(r"/kaggle/input/tabular-playground-series-feb-2021/train.csv")
sample_submission=pd.read_csv(r"/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv")

In [ ]:
display(train.head())
display(test.head())
display(sample_submission.head())

In [ ]:
print(train.columns)
print(test.columns)

In [ ]:
feature_cols = train.drop(['id','target'], axis=1).columns
feature_cols

In [ ]:
train_con_data = train[feature_cols].select_dtypes(include=[np.number])
train_cat_data = train[feature_cols].select_dtypes(exclude=[np.number])
display(train_con_data.head())
display(train_cat_data.head())

In [ ]:
train_indexs = train.index
test_indexs = test.index


df =  pd.concat(objs=[train, test], axis=0).reset_index(drop=True)
df = df.drop('id', axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
f,ax=plt.subplots(2,1,figsize=(12,6))

sns.kdeplot(data=train, x="target",fill=True,ax=ax[0])
sns.boxplot(x='target', data=train, orient='h', ax=ax[1]);

In [ ]:
Q1 = np.percentile(train[['target']], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(train[['target']], 75,
                   interpolation = 'midpoint')
IQR=Q3-Q1

upper = np.where(train['target'] >= (Q3+1.5*IQR))
lower = np.where(train['target'] <= (Q1-1.5*IQR))

train.drop(upper[0], inplace = True)
train.drop(lower[0], inplace = True)

In [ ]:
f, axes = plt.subplots(4,4, figsize=(12, 14))
f.suptitle('Distribution of Features', fontsize=16)

for index, column in enumerate(train_con_data.columns):
    i,j = (index // 4, index % 4)
    g = sns.kdeplot(train_con_data[column], color="m", shade=True, label="%.2f"%(train_con_data[column].skew()), ax=axes[i,j])
    g = g.legend(loc="best")

In [ ]:
f,ax=plt.subplots(figsize=(14,6))

sns.heatmap(train_con_data.corr(),annot=True,ax=ax)

In [ ]:
f,ax=plt.subplots(3,4,figsize=(12,12))
for index, column in enumerate(train_cat_data.columns):
    i,j = (index // 4, index % 4)
    sns.countplot(x=column, data=train_cat_data,ax=ax[i,j])

In [ ]:
for feature in train_cat_data.columns:
    le = LabelEncoder()
    le.fit(train[feature])
    train[feature] = le.transform(train[feature])
    test[feature] = le.transform(test[feature])
display(train.head())
display(test.head())

In [ ]:
x = train[feature_cols]
y = train[['target']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
#Normalization: (X-X.min())/(X.max()-X.min())
#Standardization: (X-X.mean())/X.std()


scaler_x = MinMaxScaler().fit(x_train)
scaler_y = MinMaxScaler().fit(y_train)

In [ ]:
x_train_sc = scaler_x.transform(x_train)
x_test_sc = scaler_x.transform(x_test)
y_train_sc = scaler_y.transform(y_train)
y_test_sc = scaler_y.transform(y_test)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(x_train_sc, y_train_sc)
y_pred_sc = lr_model.predict(x_test_sc)

In [ ]:
mae = mean_absolute_error(y_test_sc, y_pred_sc)
rmse = np.sqrt(mean_squared_error(y_test_sc, y_pred_sc))

print('MAE = ', mae.round(4))
print('RMSE = ', rmse.round(4))

In [ ]:
y_test_inv = scaler_y.inverse_transform(y_test_sc.reshape(-1,1))
y_pred_inv = scaler_y.inverse_transform(y_pred_sc.reshape(-1,1))

actual_mae = mean_absolute_error(y_test_inv, y_pred_inv)
actual_rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

print('Actual MAE = ', actual_mae)
print('Actual RMSE = ', actual_rmse)

In [ ]:
test_actual = test.drop('id', axis=1)

test_actual_scaler_x = MinMaxScaler().fit(test_actual)
test_actual_scaler_x_sc = test_actual_scaler_x.transform(test_actual)

test_actual_scaler_y_sc = lr_model.predict(test_actual_scaler_x_sc)

actual_rmse = np.sqrt(mean_squared_error(sample_submission['target'], test_actual_scaler_y_sc))
actual_rmse

In [ ]:
Result = pd.DataFrame()
Result['id']=test.id
Result['target']=test_actual_scaler_y_sc
Result